In [1]:
import torch
import torch.nn.functional as F

from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso

import os
import json

C:\Users\91900\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'captum'

In [5]:
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
#from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter
import pandas as pd
import numpy as np
from IPython.core.display import HTML, display

ImportError: DLL load failed while importing _torchtext: The specified procedure could not be found.

In [4]:
!pip install torchtext

     ---------------------------------------- 2.2/2.2 MB 7.9 MB/s eta 0:00:00
     -------------------------------------- 161.8/161.8 MB 5.2 MB/s eta 0:00:00


In [5]:
df=pd.read_csv(r"C:\Users\91900\Capstone\cap-main\irmapreprocessed.csv")

In [6]:
df

,id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,...,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,905625009430949888,905625009430949888_0,informative,0.6721,informative,1.0000,infrastructure_and_utility_damage,0.6721,infrastructure_and_utility_damage,1.0000,...,316,756,4,Jamaica,Cecile,slimjamaican,40303,NaN,NaN,False
1,905625059326353409,905625059326353409_0,informative,0.6545,informative,1.0000,other_relevant_information,0.6545,other_relevant_information,1.0000,...,53361,1999,408,"Washington, DC",Steve Rudin 7News,SteveRudin7News,30933,NaN,http://wjla.com/station/people/steve-rudin,True
2,905625064451833856,905625064451833856_0,informative,0.6629,informative,0.6392,infrastructure_and_utility_damage,0.6629,other_relevant_information,0.6392,...,34,179,0,Toronto,Christine 🌸,kiki_chiquita,490,NaN,NaN,False
3,905625073628975106,905625073628975106_0,informative,1.0000,informative,1.0000,other_relevant_information,1.0000,other_relevant_information,1.0000,...,597,2962,9,"Florida, USA",Laura Dill,Perspectvz,62816,NaN,NaN,False
4,905625088300654592,905625088300654592_0,informative,1.0000,informative,1.0000,other_relevant_information,1.0000,other_relevant_information,1.0000,...,2220,4298,68,Chile,Edhagadol,edhagadol,59091,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,910270036861886465,910270036861886465_0,informative,1.0000,informative,1.0000,other_relevant_information,1.0000,infrastructure_and_utility_damage,1.0000,...,406,243,8,NaN,On Weather,OnWeather1,71804,NaN,NaN,False
3516,910270462822895617,910270462822895617_0,informative,1.0000,not_informative,0.3779,other_relevant_information,1.0000,not_humanitarian,0.3779,...,591,61,23,"Kingsland, GA",Camden County EMA,CamdenCountyEMA,4635,NaN,https://www.camdencountyga.gov/81/Emergency-Ma...,False
3517,910270554812317697,910270554812317697_0,informative,0.6376,informative,0.6445,other_relevant_information,0.6376,other_relevant_information,0.6445,...,1090,709,29,(223/717),Nikhil Menon,multimenon,41946,NaN,https://nikhilmenon.in,False
3518,910270595262291968,910270595262291968_0,not_informative,0.6667,not_informative,1.0000,not_humanitarian,0.6667,not_humanitarian,1.0000,...,1533,3859,183,"Wewoka, OK",Jake Davis,WewokaChronicle,208925,NaN,http://writers.work/jakedavis,False


In [6]:
df2=pd.read_csv(r"C:\Users\91900\Cap\model1.csv")

NameError: name 'pd' is not defined

In [7]:
# cwd="/Users/Ishita/Workspace/PES/sem6/capstone/Datasets"
ls=[]
# for filename in os.listdir(cwd):
#     df=pd.read_csv(filename)
ls.append(df)
res=pd.concat(ls)
lsval=[]
def fun(x):
    lsval.append((1 if x['text_info']=='informative' else 0,x['text']))
    
res.apply(fun,axis=1)
len(lsval)

3520

In [ ]:
# cwd="/Users/Ishita/Workspace/PES/sem6/capstone/Datasets"
ls1=[]
# for filename in os.listdir(cwd):
#     df=pd.read_csv(filename)
ls1.append(df2)
res1=pd.concat(ls1)
lsval1=[]
def fun(x):
    lsval1.append(x['text'])
    
res.apply(fun,axis=1)
len(lsval)

In [8]:
lsval

[(1,
  'island barbuda liter water hurrican irma http co xrolftgz http co f w dn'),
 (1, 'pm hurrican irma updat weather http co mflgiknbyy'),
 (1,
  'hurrican irma destroy upward barbuda offici tell local media http co xmwdbzxcvz http co pjvuoe x'),
 (1, 'pm track updat hurrican irma flwx gawx jaxwx http co lv cztlvst'),
 (1, 'pm advisori hurrican irma firstalertwx http co zixrhp qrd'),
 (1, 'releas statement hurrican irma http co rpfq vdumo'),
 (1,
  'jetblu sell ticket peopl escap hurrican irma http co fv blkzidj http co sjirzmv q'),
 (1,
  'terrifi reason fear miami surviv hurricaneirma http co f awvm gi http co p lk fh dj'),
 (1, 'pray hurrican irma skip sweet villag la cola http co r bv jk'),
 (1, 'pray hurrican irma skip sweet villag la cola http co r bv jk'),
 (1, 'pray hurrican irma skip sweet villag la cola http co r bv jk'),
 (1, 'pray hurrican irma skip sweet villag la cola http co r bv jk'),
 (1,
  'puerto rico could without power month hurrican irma http co tm lpk http co

In [9]:
ag_train,ag_val=lsval[:2816],lsval[2816:]
tokenizer = get_tokenizer('basic_english')
word_counter = Counter()
for (label, line) in ag_train:
    word_counter.update(tokenizer(line))
voc = Vocab(word_counter)

print('Vocabulary size:', len(voc))

num_class = len(set(label for label, _ in ag_train))
print('Num of classes:', num_class)

Vocabulary size: 10223
Num of classes: 2


In [10]:
print(len(ag_val))

704


In [11]:
class EmbeddingBagModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, num_class)

    def forward(self, inputs, offsets):
        embedded = self.embedding(inputs, offsets)
        return self.linear(embedded)

In [12]:
BATCH_SIZE = 64

def collate_batch(batch):
    labels = torch.tensor([label  for label, _ in batch]) 
    text_list = [tokenizer(line) for _, line in batch]
    
    # flatten tokens across the whole batch
    text = torch.tensor([voc[t] for tokens in text_list for t in tokens])
    # the offset of each example
    offsets = torch.tensor(
        [0] + [len(tokens) for tokens in text_list][:-1]
    ).cumsum(dim=0)

    return labels, text, offsets

train_loader = DataLoader(ag_train, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(ag_val, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=collate_batch)

In [13]:
EPOCHS = 20
EMB_SIZE = 128
CHECKPOINT = './models/embedding_bag_ag_news.pt'
USE_PRETRAINED = False  # change to False if you want to retrain your own model

def train_model(train_loader, val_loader):
    model = EmbeddingBagModel(len(voc), EMB_SIZE, num_class)
    
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(1, EPOCHS + 1):      
        # training
        model.train()
        total_acc, total_count = 0, 0
        
        for idx, (label, text, offsets) in enumerate(train_loader):
            optimizer.zero_grad()
            predited_label = model(text, offsets)
            loss(predited_label, label).backward()
            optimizer.step()
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

            if (idx + 1) % 500 == 0:
                print('epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}'.format(
                    epoch, idx + 1, len(train_loader), total_acc / total_count
                ))
                total_acc, total_count = 0, 0       
        
        # evaluation
        model.eval()
        total_acc, total_count = 0, 0

        with torch.no_grad():
            for label, text, offsets in val_loader:
                predited_label = model(text, offsets)
                total_acc += (predited_label.argmax(1) == label).sum().item()
                total_count += label.size(0)

        print('-' * 59)
        print('end of epoch {:3d} | valid accuracy {:8.3f} '.format(epoch, total_acc / total_count))
        print('-' * 59)
    
    #torch.save(model, CHECKPOINT)
    return model
        
eb_model = torch.load(CHECKPOINT) if USE_PRETRAINED else train_model(train_loader, val_loader)

-----------------------------------------------------------
end of epoch   1 | valid accuracy    0.791 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   2 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   3 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   4 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   5 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   6 | valid accuracy    0.793 
-----------------------------------------------------------
----------------

In [15]:
test_label = 1
#test_line = ("RT @UNFspinnaker: ' To me Puerto Rico is more than just a paradise island–it’s my home ' ")
# test_line= ("RT @businessinsider: JetBlue is selling $99 tickets so that people can escape Hurricane Irma https://t.co/fv3BLkZIdj https://t.co/SjiRZMV07q")
test_line =("record break hurrican irma wreak havoc across caribbean apocalypt http co xloq fcbrf trend http co etcftjcxl")
test_labels, test_text, test_offsets = collate_batch([(test_label, test_line)])

probs = F.softmax(eb_model(test_text, test_offsets), dim=1).squeeze(0)
print('Prediction probability:', round(probs[test_labels[0]].item(), 4))

Prediction probability: 0.9069


In [16]:
# remove the batch dimension for the embedding-bag model
def forward_func(text, offsets):
    return eb_model(text.squeeze(0), offsets)

# encode text indices into latent representations & calculate cosine similarity
def exp_embedding_cosine_distance(original_inp, perturbed_inp, _, **kwargs):
    original_emb = eb_model.embedding(original_inp, None)
    perturbed_emb = eb_model.embedding(perturbed_inp, None)
    distance = 1 - F.cosine_similarity(original_emb, perturbed_emb, dim=1)
    return torch.exp(-1 * (distance ** 2) / 2)

# binary vector where each word is selected independently and uniformly at random
def bernoulli_perturb(text, **kwargs):
    probs = torch.ones_like(text) * 0.5
    return torch.bernoulli(probs).long()

# remove absenst token based on the intepretable representation sample
def interp_to_input(interp_sample, original_input, **kwargs):
    return original_input[interp_sample.bool()].view(original_input.size(0), -1)

lasso_lime_base = LimeBase(
    forward_func, 
    interpretable_model=SkLearnLasso(alpha=0.08),
    similarity_func=exp_embedding_cosine_distance,
    perturb_func=bernoulli_perturb,
    perturb_interpretable_space=True,
    from_interp_rep_transform=interp_to_input,
    to_interp_rep_transform=None
)

In [17]:
attrs = lasso_lime_base.attribute(
    test_text.unsqueeze(0),
    target=test_labels,
    additional_forward_args=(test_offsets,),
    n_samples=32000,
    show_progress=True
).squeeze(0)

print('Attribution range:', attrs.min().item(), 'to', attrs.max().item())

Lime Base attribution: 100%|██████████| 32000/32000 [00:31<00:00, 1007.69it/s]


Attribution range: 0.0 to 0.6954545378684998


In [18]:
def show_text_attr(attrs):
    rgb = lambda x: '255,0,0' if x < 0 else '0,255,0'
    alpha = lambda x: abs(x) ** 0.5
    token_marks = [
        f'<mark style="background-color:rgba({rgb(attr)},{alpha(attr)})">{token}</mark>'
        for token, attr in zip(tokenizer(test_line), attrs.tolist())
    ]
    
    display(HTML('<p>' + ' '.join(token_marks) + '</p>'))
    
show_text_attr(attrs)